# Prueba Practica 4
## Nombre: Alex Reinoso
## Materia: Simulacion
## Fecha: 02/07/2022

## Instalar e importar

In [1]:
pip install simpy

In [8]:
import simpy
from scipy import stats 
from random import expovariate,seed
import random

## Variables

In [49]:
CANT_CAJEROS = 3
CANT_CLIENTES = 100
MEDIA = 3 
TIEMPO_SIMULACION = 720

## Simulacion

In [62]:
tiempo = []
class Banco(object):
  def __init__(self, environment, num_cajeros):
    self.env = environment
    self.cajeros = simpy.Resource(environment, num_cajeros)
  
  def atender_cliente(self, cliente):
    # aqui esta el tiempo que se necesita para atender un cliente con 
    #la distribucion exponencial y la media dada de 3 minutos
    yield self.env.timeout(expovariate(1.0/MEDIA ))

def llegada_clientes(env, nombre, banco):
  llegada = round(env.now,2)  
  print('Llega cliente: %s a la hora %.2f.' % (nombre, env.now))
  with banco.cajeros.request() as cajero:
    yield cajero
    yield env.process(banco.atender_cliente(nombre))
    final = round(env.now,2)  
    tiempo.append(final-llegada)
    print('Cliente [%s] fue atendido a las %.2f.'%(nombre, env.now))
    

def ejecutar_simulacion(env, num_cajeros):
  banco = Banco(env, num_cajeros)
  for i in range(5):
    env.process(llegada_clientes(env, 'Cliente-%d'%(i+1),banco))
  while True:
    yield env.timeout(random.randint(5, 10))
    i+=1
    env.process(llegada_clientes(env, 'Cliente-%d'%(i+1), banco))
random.seed(77)
env=simpy.Environment()
# Aqui estoy mandando a ejecutar la simulacion con la cantidad de cajeros
# previamente definida
env.process(ejecutar_simulacion(env, CANT_CAJEROS))
env.run(until = TIEMPO_SIMULACION)

Llega cliente: Cliente-1 a la hora 0.00.
Llega cliente: Cliente-2 a la hora 0.00.
Llega cliente: Cliente-3 a la hora 0.00.
Llega cliente: Cliente-4 a la hora 0.00.
Llega cliente: Cliente-5 a la hora 0.00.
Cliente [Cliente-2] fue atendido a las 0.83.
Cliente [Cliente-1] fue atendido a las 1.18.
Cliente [Cliente-4] fue atendido a las 1.19.
Cliente [Cliente-5] fue atendido a las 3.12.
Cliente [Cliente-3] fue atendido a las 5.19.
Llega cliente: Cliente-6 a la hora 7.00.
Llega cliente: Cliente-7 a la hora 16.00.
Cliente [Cliente-7] fue atendido a las 16.47.
Llega cliente: Cliente-8 a la hora 22.00.
Cliente [Cliente-8] fue atendido a las 25.25.
Cliente [Cliente-6] fue atendido a las 31.32.
Llega cliente: Cliente-9 a la hora 32.00.
Llega cliente: Cliente-10 a la hora 37.00.
Cliente [Cliente-9] fue atendido a las 37.71.
Llega cliente: Cliente-11 a la hora 42.00.
Cliente [Cliente-10] fue atendido a las 42.41.
Cliente [Cliente-11] fue atendido a las 42.57.
Llega cliente: Cliente-12 a la hora 50.

## Resultados

In [63]:
print('Tiempo requerido para atender %.1f clientes es %.2f minutos'%(CANT_CLIENTES, round(sum(tiempo), 2)))

Tiempo requerido para atender 100.0 clientes es 315.67 minutos


In [64]:
print('Tiempo requerido para atender %.1f clientes es %.2f horas'%(CANT_CLIENTES, (round(sum(tiempo), 2)/60)))

Tiempo requerido para atender 100.0 clientes es 5.26 horas
